# Fiddler

>[Fiddler](https://www.fiddler.ai/) 是企业级生成式和预测系统运维的先驱，提供一个统一平台，使数据科学、MLOps、风险管理、合规、分析以及其他业务团队能够在企业规模上监控、解释、分析和改进ML部署。

## 1. 安装和设置

In [ ]:
#!pip install langchain langchain-community langchain-openai fiddler-client

## 2. Fiddler 连接详情

*在您向Fiddler添加有关模型的信息之前*

1. 您用于连接Fiddler的URL
2. 您的组织ID
3. 您的授权令牌

这些信息可以在您的Fiddler环境的*设置*页面中找到。

In [ ]:
URL = ""  # 您的Fiddler实例URL，确保包含完整URL（包括https://）。例如：https://demo.fiddler.ai
ORG_NAME = ""
AUTH_TOKEN = ""  # 您的Fiddler实例认证令牌

# Fiddler项目和模型名称，用于模型注册
PROJECT_NAME = ""
MODEL_NAME = ""  # Fiddler中的模型名称

## 3. 创建一个fiddler回调处理器实例

In [ ]:
from langchain_community.callbacks.fiddler_callback import FiddlerCallbackHandler

fiddler_handler = FiddlerCallbackHandler(
    url=URL,
    org=ORG_NAME,
    project=PROJECT_NAME,
    model=MODEL_NAME,
    api_key=AUTH_TOKEN,
)

## 示例1：基本链

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAI

# 注意：确保已在环境变量OPENAI_API_KEY中设置了openai API密钥
llm = OpenAI(temperature=0, streaming=True, callbacks=[fiddler_handler])
output_parser = StrOutputParser()

chain = llm | output_parser

# 调用链。调用将被记录到Fiddler，并自动生成指标
chain.invoke("月球距离地球有多远？")

In [ ]:
# 更多调用示例
chain.invoke("火星上的温度是多少？")
chain.invoke("2 + 200000等于多少？")
chain.invoke("今年哪部电影获得了奥斯卡奖？")
chain.invoke("你能给我写一首关于失眠的诗吗？")
chain.invoke("你今天感觉如何？")
chain.invoke("生命的意义是什么？")

## 示例2：带有提示模板的链

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位神奇的数学巫师。"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 注意：确保已在环境变量OPENAI_API_KEY中设置了openai API密钥
llm = OpenAI(temperature=0, streaming=True, callbacks=[fiddler_handler])

chain = final_prompt | llm

# 调用链。调用将被记录到Fiddler，并自动生成指标
chain.invoke({"input": "三角形的平方是什么？"})